In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from linearmodels import IV2SLS
from rpy2.robjects import Formula, pandas2ri
from rpy2.robjects.packages import importr
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from statsmodels.tools import add_constant
import warnings
warnings.filterwarnings('ignore')

In [2]:
pandas2ri.activate()

In [3]:
data2 = pd.read_csv("./justice_name/model_data.csv")
data2.head()

,Unnamed: 0,file_name,court,title,petitioner,respondent,doc_date,judgment_order,Juridiction,Judge,...,Len,Year,crime_rate,facts_vector,judgement_vector,Similar_facts_index,Similar_judgement_index,past_3_judgmenent_vec,gov_resp,gov_pet
0,0,34613760,Delhi High Court,"Vishal Yadav vs State Of U.P. on 2 April, 2014",Vishal Yadav,State Of U.P.,2014-04-02,0,delhi,gita mittal,...,1697474,2014,92.5,[-0.07737926 -1.15306365 -1.17525125 3.510302...,[-0.79722774 -0.46005258 -1.29749489 -0.701790...,"[17633, 17634, 21144]","[27578, 17677, 15281]","[array([-0.70901322, -0.48732516, -1.05696547,...",1.0,0.0
1,1,57145403,Delhi High Court,State Through Reference vs Ram Singh & Ors. on...,State Through Reference,Ram Singh & Ors.,2014-03-13,1,delhi,reva khetrapal,...,625268,2014,92.5,[ 0.39603654 0.15277775 -2.79244351 2.946386...,[-2.59295011 -0.19979216 -1.92205489 -0.626127...,"[79, 13, 1100]","[27572, 26383, 23265]","[array([ 0.17494856, 1.18929636, -0.92777139,...",0.0,0.0
2,2,33817559,Delhi High Court,Santoshanand Avdoot @ Ghanshyam ... vs State o...,Santoshanand Avdoot @ Ghanshyam ...,State,2014-08-14,1,delhi,s. p. garg,...,285226,2014,92.5,[ 0.56868905 -2.24206519 -1.48725712 2.021034...,[-0.94152963 -1.640522 -2.38711333 -1.751034...,"[7955, 7956, 7954]","[8418, 17640, 1158]","[array([ 0.16465889, -0.70015794, -0.1884212 ,...",1.0,0.0
3,3,33863247,Delhi High Court,"Prabhat Singh vs State on 8 July, 2014",Prabhat Singh,State,2014-07-08,1,delhi,sunita gupta,...,165015,2014,92.5,[ 0.17429775 -0.17024837 -0.66277361 0.528286...,[-0.10857403 -0.83127654 -0.55613422 -3.057985...,"[16, 42, 17678]","[17723, 13, 22147]","[array([-0.584759 , -1.07153809, -2.04350471,...",1.0,0.0
4,4,27729615,Delhi High Court,Manohar Lal Sharma Advocate vs The Principal S...,Manohar Lal Sharma Advocate,"The Principal Secretary, Prime ...",2014-08-13,1,delhi,badar,...,112077,2014,92.5,[-1.98948753 -1.2194823 -1.42316091 3.738958...,[-2.09565043 0.96318555 -0.98214722 0.044335...,"[5, 15272, 22627]","[5, 15272, 23211]","[array([-1.36835074, -1.08036482, -3.70619917,...",0.0,0.0


In [6]:
data2.columns

Index(['Unnamed: 0', 'file_name', 'court', 'title', 'petitioner', 'respondent',
       'doc_date', 'judgment_order', 'Juridiction', 'Judge', 'Text', 'harsh',
       'Text_fact', 'Text_judgement', 'Len', 'Year', 'crime_rate',
       'facts_vector', 'judgement_vector', 'Similar_facts_index',
       'Similar_judgement_index', 'past_3_judgmenent_vec', 'gov_resp',
       'gov_pet'],
      dtype='object')

In [10]:
data2[['file_name','court',"title",'petitioner','respondent','doc_date','judgment_order','Juridiction','Judge','facts_vector','past_3_judgmenent_vec','harsh','crime_rate']].sample(5)

,file_name,court,title,petitioner,respondent,doc_date,judgment_order,Juridiction,Judge,facts_vector,past_3_judgmenent_vec,harsh,crime_rate
18812,86522134,Madras High Court,Muruganandham @ Murugan vs The State Rep. By o...,Muruganandham @ Murugan,The State Rep. By,2016-11-15,1,chennai,s.nagamuthu,[ 0.4782986 -1.97604847 -1.62539315 -1.065090...,"[array([-1.94472945, 1.17477465, 0.0512206 ,...",1,15.6
17835,558976,Delhi High Court,Ibrahim @ Guddu vs The State Of Nct Of Delhi o...,Ibrahim @ Guddu,The State Of Nct Of Delhi,2011-10-13,0,delhi,badar durrez ahmed,[-0.73212075 -0.83306962 -0.51677364 0.264510...,"[array([-4.49652624, -0.73144668, -3.69586515,...",0,37.0
9685,14458059,Allahabad High Court,Kallu vs State Of U.P. And Another on 1 Novemb...,Kallu,State Of U.P. And Another,2019-11-01,0,allahabad,virendra kumar srivastava hi,[-0.18313703 0.71127021 0.6385504 -0.567722...,"[array([-0.60532218, -0.6775474 , -0.30318025,...",0,24.6
8552,149901158,Bombay High Court,Sandeep Rangrao Kumare vs State Of Maharashtra...,Sandeep Rangrao Kumare,"State Of Maharashtra, Through Its ...",2016-06-06,1,bombay,B.R. Gavai,[ 0.08335043 -0.09803686 -2.15783548 2.402804...,"[array([-0.2732237 , -0.37379491, -0.39326555,...",1,35.2
4991,192052785,Madras High Court,R.Sundaramoorthy vs State Represented By on 29...,R.Sundaramoorthy,State Represented By,2019-07-29,1,chennai,v.bharathidasan,[-1.20916271 -0.21619818 -0.43000326 -0.916005...,"[array([ 3.38784337e-01, -6.74185082e-02, -6.4...",0,14.4


In [7]:
ivpack = importr("ivpack")
aer = importr("AER")
r_base = importr("base")

In [8]:
data = pd.read_csv("./justice_name/IV_data.csv")

In [9]:
data.judgement_pred.value_counts()

0    24094
1     6958
Name: judgement_pred, dtype: int64

In [10]:
data.head()

,Jud_0,Jud_1,Jud_2,Jud_3,Jud_4,Jud_5,Jud_6,Jud_7,Jud_8,Jud_9,...,fact_emb_44,fact_emb_45,fact_emb_46,fact_emb_47,fact_emb_48,fact_emb_49,judgement_pred,gov_pet,gov_resp,crime_rate
0,-0.709013,-0.487325,-1.056965,0.818804,-0.798959,1.001305,0.691063,0.193017,-0.804408,0.633024,...,2.563970,0.277645,0.018563,1.128461,1.312803,-2.323720,0,0.0,1.0,92.5
1,0.174949,1.189296,-0.927771,0.257538,-0.298800,1.077674,-0.749863,-0.397366,0.008756,-0.734707,...,1.598934,0.472292,0.188702,-1.014126,2.751991,-1.364089,1,0.0,0.0,92.5
2,0.164659,-0.700158,-0.188421,-0.743105,0.681166,1.100095,0.149085,-0.199905,-0.679346,0.046218,...,3.449621,0.274740,-1.984469,-1.442404,-0.851224,1.276552,0,0.0,1.0,92.5
3,-0.584759,-1.071538,-2.043505,0.484414,0.151943,-0.082056,0.598946,-0.408808,0.550904,-0.182296,...,0.914630,-0.420852,-0.492278,-2.264972,3.601571,0.398855,0,0.0,1.0,92.5
4,-1.368351,-1.080365,-3.706199,0.516378,-0.244941,1.347507,1.047686,1.546216,3.523230,-1.889053,...,3.059956,1.365849,3.039275,-0.088997,0.342930,3.686197,1,0.0,0.0,92.5


In [47]:
df = pd.read_csv("./justice_name/fe.csv")

In [48]:
df.Juridiction.unique()

array(['delhi', 'allahabad', 'chennai', 'bombay', 'kerala'], dtype=object)

In [12]:
df['Year'] = [str(x) for x in df.Year]

In [13]:
data = pd.concat((data,df),axis=1)

In [14]:
#Insturmental Variable Columns - Embeddings of judgements by that Judge in past 3 cases
instru_var_cols = data.columns[:150]
instru_data = data[instru_var_cols]
instru_data.head()

,Jud_0,Jud_1,Jud_2,Jud_3,Jud_4,Jud_5,Jud_6,Jud_7,Jud_8,Jud_9,...,Jud_140,Jud_141,Jud_142,Jud_143,Jud_144,Jud_145,Jud_146,Jud_147,Jud_148,Jud_149
0,-0.709013,-0.487325,-1.056965,0.818804,-0.798959,1.001305,0.691063,0.193017,-0.804408,0.633024,...,-0.058398,-1.668045,1.188825,0.788128,-1.885749,0.278859,-0.054479,-2.100279,1.034174,-1.231385
1,0.174949,1.189296,-0.927771,0.257538,-0.298800,1.077674,-0.749863,-0.397366,0.008756,-0.734707,...,-2.108963,-1.870219,-1.867665,-0.657096,-2.281700,0.161461,-1.713752,-0.332595,1.556750,0.761622
2,0.164659,-0.700158,-0.188421,-0.743105,0.681166,1.100095,0.149085,-0.199905,-0.679346,0.046218,...,-0.266416,0.997326,-0.818119,0.844203,-0.603537,1.588145,-0.283121,-0.996231,0.791162,0.468696
3,-0.584759,-1.071538,-2.043505,0.484414,0.151943,-0.082056,0.598946,-0.408808,0.550904,-0.182296,...,0.193194,0.004010,-2.751466,-3.539840,-0.875378,0.360259,-0.244792,-0.170450,1.971528,1.845870
4,-1.368351,-1.080365,-3.706199,0.516378,-0.244941,1.347507,1.047686,1.546216,3.523230,-1.889053,...,-1.688663,-4.064988,0.504453,-0.026259,0.093963,2.178278,-1.309577,-2.458808,1.464417,-0.882024


In [15]:
#Endogenous Variable = Judgment - Harsh or Lenient
endogenous = data['judgement_pred']
endogenous.head()

0    0
1    1
2    0
3    0
4    1
Name: judgement_pred, dtype: int64

In [39]:
#Exogenous - Emebddings of facts of the particular Case and Govt-Petitioner or Respondent Flag

exog_cols = list(data.columns[151:-1])
exog_cols.remove('judgement_pred')
exog_cols.remove('crime_rate')
exogenous = data[exog_cols]
exogenous.head()

,fact_emb_0,fact_emb_1,fact_emb_2,fact_emb_3,fact_emb_4,fact_emb_5,fact_emb_6,fact_emb_7,fact_emb_8,fact_emb_9,...,fact_emb_43,fact_emb_44,fact_emb_45,fact_emb_46,fact_emb_47,fact_emb_48,fact_emb_49,gov_pet,gov_resp,Juridiction
0,-0.077379,-1.153064,-1.175251,3.510302,2.280033,-0.964784,-2.382688,0.655629,0.204327,-2.628231,...,-2.968399,2.563970,0.277645,0.018563,1.128461,1.312803,-2.323720,0.0,1.0,delhi
1,0.396037,0.152778,-2.792444,2.946386,0.100429,-0.727175,-1.564026,-0.456334,-1.848460,-0.769046,...,-2.717016,1.598934,0.472292,0.188702,-1.014126,2.751991,-1.364089,0.0,0.0,delhi
2,0.568689,-2.242065,-1.487257,2.021035,0.655754,-0.120980,-3.916295,-0.261597,0.625245,-2.353674,...,-3.404712,3.449621,0.274740,-1.984469,-1.442404,-0.851224,1.276552,0.0,1.0,delhi
3,0.174298,-0.170248,-0.662774,0.528286,-0.055306,-0.349290,-3.064025,1.223578,-0.028150,-0.338014,...,-1.674878,0.914630,-0.420852,-0.492278,-2.264972,3.601571,0.398855,0.0,1.0,delhi
4,-1.989488,-1.219482,-1.423161,3.738959,5.863144,-1.498037,1.876165,-1.304972,-3.018544,1.998867,...,1.839812,3.059956,1.365849,3.039275,-0.088997,0.342930,3.686197,0.0,0.0,delhi


In [40]:
outcome = data['crime_rate']

In [45]:
tsls_fit = IV2SLS(
    dependent=outcome,
    exog=exogenous,
    endog=endogenous,
    instruments=instru_data
).fit(cov_type='clustered')

In [46]:
print(tsls_fit)

                          IV-2SLS Estimation Summary                          
Dep. Variable:             crime_rate   R-squared:                      0.9300
Estimator:                    IV-2SLS   Adj. R-squared:                 0.9298
No. Observations:               31052   F-statistic:                 8.267e+05
Date:                Thu, May 13 2021   P-value (F-stat)                0.0000
Time:                        15:24:11   Distribution:                 chi2(57)
Cov. Estimator:             clustered                                         
                                                                              
                                  Parameter Estimates                                  
                     Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
---------------------------------------------------------------------------------------
fact_emb_0             -1.6731     0.1215    -13.776     0.0000     -1.9112     -1.4351
fact_emb_1      

In [29]:
print(tsls_fit.first_stage)

      First Stage Estimation Results     
                           judgement_pred
-----------------------------------------
R-squared                          0.3209
Partial R-squared                  0.0414
Shea's R-squared                   0.0414
Partial F-statistic                1183.9
P-value (Partial F-stat)           0.0000
Partial F-stat Distn            chi2(150)
==========================   ============
fact_emb_0                        -0.0108
                                (-2.7498)
fact_emb_1                        -0.0056
                                (-1.4806)
fact_emb_2                         0.0007
                                 (0.1785)
fact_emb_3                        -0.0076
                                (-1.8561)
fact_emb_4                        -0.0255
                                (-6.1146)
fact_emb_5                         0.0137
                                 (3.3774)
fact_emb_6                         0.0141
                                 (

In [30]:
outcome1, outcome2, treatment1, treatment2, instrument1, instrument2 = train_test_split(
    outcome, endogenous, instru_data, train_size=0.5
)
folds = [(outcome1, treatment1, instrument1), (outcome2, treatment2, instrument2)]

In [31]:

ols_fit = sm.OLS(endog=folds[0][1], exog=add_constant(folds[0][2])).fit()

In [32]:
def calculate_r2(predicted, truth):
    """Calculate R^2 of predicted and truth with 1 - (truth - predicted)^2 / truth.std()^2"""
    return 1 - ((truth - predicted) ** 2).mean() / truth.std() ** 2


calculate_r2(ols_fit.predict(add_constant(folds[1][2])), folds[1][1])

0.06792209210794242

In [35]:
def fit_predict_clf(clf, fold, predict_fold):
    """Fit clf on fold and predict on predict_fold"""
    outcome, treatment, instrument = fold
    clf.fit(instrument, treatment)
    return clf.predict(predict_fold[2])


def two_fold_cross_fit(folds):
    """Perform cross-fitting"""
    fold1, fold2 = folds

    predicted2 = fit_predict_clf(clf, fold1, fold2)
    predicted1 = fit_predict_clf(clf, fold2, fold1)

    return (
        predicted1,
        predicted2,
        calculate_r2(predicted1, fold1[1]),
        calculate_r2(predicted2, fold2[1]),)
    

In [38]:
#predicted1, predicted2, rsq1, rsq2 = two_fold_cross_fit(folds)
#print(rsq1, rsq2)